In [24]:
#imports
import numpy as np
import pandas as pd 
import pydicom
import os
import sys
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [3]:
#path constants
ROOT_DIR = '../'
DATA_DIR = 'data/train_images/'
DEST_DIR = 'raw_data/'

In [4]:
def determine_orientation(orientation_arr: list):
    '''
    Conversion was found in a comment under 
    https://stackoverflow.com/questions/34782409/understanding-dicom-image-attributes-to-get-axial-coronal-sagittal-cuts
    '''
    coronal_plane  = [1, 0, 0, 0, 0, -1]
    sagittal_plane = [0, 1, 0, 0, 0, -1]
    axial_plane    = [1, 0, 0, 0, 1,  0]
    
    rounded_arr = [round(i) for i in orientation_arr]
    if rounded_arr == coronal_plane:
        return 'coronal'
    elif rounded_arr == sagittal_plane:
        return 'sagittal'
    elif rounded_arr == axial_plane:
        return 'axial'
    else:
        return 'indeterminate'

In [5]:
#patient directories
studies = os.listdir(os.path.join(ROOT_DIR, DATA_DIR))
studies.sort()

#metadata dataframe
metadata_df = pd.DataFrame(columns=['study_id',
                                    'series_id',
                                    'orientation',
                                    'dim_x',
                                    'dim_y',
                                    'pixel_dim_x',
                                    'pixel_dim_y',
                                    'rows',
                                    'columns',
                                    'num_slices',
                                    'slice_spacing',
                                    'slice_thickness',
                                    'series_description',
                                    'patient_position',
                                    'image_position',
                                    'image_orientation',
                                    'photometric_interpretation',
                                    'scan_dim_x_mm',
                                    'scan_dim_y_mm',
                                    'scan_dim_z_mm'])

per_slice_metadata_columns = [
    'ContentDate',
    'ContentTime',
    'SliceThickness',
    'SpacingBetweenSlices',
    'SliceLocation',
    'InstanceNumber',
    'ImagePositionPatient',
    'ImageOrientationPatient',
    'PixelSpacing',
    'WindowCenter',
    'WindowWidth'
]

In [18]:
def load_scan(path: str)-> tuple[np.ndarray, list, pd.DataFrame]:
    '''
    Extracts 3d pixel data, metadata, and per-slice metadata from scan directory
    \n Code partially taken and modified from 
    \n https://www.kaggle.com/code/gzuidhof/full-preprocessing-tutorial
    '''
    #Get slices from scan
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    
    #make 3-d 
    image = np.stack([s.pixel_array for s in slices])
    
    #Get scan metadata
    study_id                   = int(str(slices[0].SeriesInstanceUID).split('.')[0])
    series_id                  = int(str(slices[0].SeriesInstanceUID).split('.')[1])
    orientation                = determine_orientation(slices[0].ImageOrientationPatient)
    dim_x                      = slices[0].pixel_array.shape[0]
    dim_y                      = slices[0].pixel_array.shape[1]
    pixel_dim_x                = float(slices[0].PixelSpacing[0])
    pixel_dim_y                = float(slices[0].PixelSpacing[1])
    rows                       = slices[0].Rows
    columns                    = slices[0].Columns
    num_slices                 = len(slices)
    slice_spacing              = float(slices[0].SpacingBetweenSlices)
    slice_thickness            = float(slices[0].SliceThickness)
    series_description         = slices[0].SeriesDescription
    patient_position           = slices[0].PatientPosition
    image_position             = slices[0].ImagePositionPatient
    image_orientation          = slices[0].ImageOrientationPatient
    photometric_interpretation = slices[0].PhotometricInterpretation
    scan_dim_x_mm              = dim_x * pixel_dim_x
    scan_dim_y_mm              = dim_y * pixel_dim_y
    scan_dim_z_mm              = num_slices * slice_spacing
    
    #put metadata in list
    scan_metadata = [
        study_id,
        series_id,
        orientation,
        dim_x,
        dim_y,
        pixel_dim_x,
        pixel_dim_y,
        rows,
        columns,
        num_slices,
        slice_spacing,
        slice_thickness,
        series_description,
        patient_position,
        image_position,
        image_orientation,
        photometric_interpretation,
        scan_dim_x_mm,
        scan_dim_y_mm,
        scan_dim_z_mm
    ]
    
    #Get per-slice metadata
    per_slice_metadata_df = pd.DataFrame(columns=per_slice_metadata_columns)
    for slice in slices:
        ContentDate = slice.ContentDate
        ContentTime = slice.ContentTime
        SliceThickness = slice.SliceThickness
        SpacingBetweenSlices = slice.SpacingBetweenSlices
        SliceLocation = slice.SliceLocation
        InstanceNumber = slice.InstanceNumber
        ImagePositionPatient = slice.ImagePositionPatient
        ImageOrientationPatient = slice.ImageOrientationPatient
        PixelSpacing = slice.PixelSpacing
        WindowCenter = slice.WindowCenter
        WindowWidth = slice.WindowWidth
        
        slice_metadata = [
            ContentDate,
            ContentTime,
            SliceThickness,
            SpacingBetweenSlices,
            SliceLocation,
            InstanceNumber,
            ImagePositionPatient,
            ImageOrientationPatient,
            PixelSpacing,
            WindowCenter,
            WindowWidth
        ]
        # pd.concat([per_slice_metadata_df, slice_metadata])
        per_slice_metadata_df.loc[len(per_slice_metadata_df)] = slice_metadata
    
    return image, scan_metadata, per_slice_metadata_df

In [26]:
ex_dir = '4249951560/1515129023'
path = os.path.join(ROOT_DIR, DATA_DIR, ex_dir)
data = load_scan(path)

In [28]:
data[2]

,ContentDate,ContentTime,SliceThickness,SpacingBetweenSlices,SliceLocation,InstanceNumber,ImagePositionPatient,ImageOrientationPatient,PixelSpacing,WindowCenter,WindowWidth
0,20240503,224045.673226,4.0,4.8,17.053982,15,"[5.2363644, -110.210915, -269.22937]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",365.0,788.0
1,20240503,224042.028734,4.0,4.8,12.253983,14,"[0.4405747, -110.18189, -269.03046]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",338.0,741.0
2,20240503,224044.232324,4.0,4.8,7.453982,13,"[-4.3552155, -110.15287, -268.83157]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",325.0,721.0
3,20240503,224045.878562,4.0,4.8,2.653982,12,"[-9.151005, -110.123856, -268.6327]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",300.0,683.0
4,20240503,224044.552510,4.0,4.8,-2.146018,11,"[-13.9467945, -110.09483, -268.43378]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",276.0,646.0
5,20240503,224040.720082,4.0,4.8,-6.946018,10,"[-18.742584, -110.06581, -268.2349]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",273.0,641.0
6,20240503,224040.375714,4.0,4.8,-11.746017,9,"[-23.538374, -110.0368, -268.03598]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",285.0,656.0
7,20240503,224040.230423,4.0,4.8,-16.546019,8,"[-28.334164, -110.007774, -267.8371]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",284.0,651.0
8,20240503,224042.099985,4.0,4.8,-21.346016,7,"[-33.12995, -109.97875, -267.6382]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",268.0,622.0
9,20240503,224044.311460,4.0,4.8,-26.146017,6,"[-37.925743, -109.94973, -267.4393]","[0.0060510393, 0.9999817, -8.917E-12, -0.04143...","[0.78125, 0.78125]",287.0,652.0
